#### Módulo 2: Machine Learning Aplicado

# 25. Práctica: Mastering Random Forest y XGBoost

#### Contenidos:
* [25.1 Dataset](#1)
* [25.2 Mastering Random Forest](#2)
* [25.3 Mastering XGBoost](#3)
* [25.4 Pruebas libres adicionales](#4)

En este notebook vamos a poner en práctica lo aprendido sobre Random Forest y XGBoost de manera un poco más exhaustiva. Estudiaremos diferentes configuraciones de hiper-parámetros y veremos cómo optimizarlos. 

## 25.1. Dataset  <a class="anchor" id="1"></a>

Vamos a trabajar sobre uno de los problemas de clasificación tratados en la práctica anterior: Pima. Se trata de un dataset con 768 ejemplos en el que el objetivo es detectar si una mujer de al menos 21 años de edad de origen indio Pima padece diabetes o no. Para ello, se utilizan las siguientes 8 variables:

1. Embarazada = Número de veces embarazada
2. Plas = Concentración de glucosa en plasma a 2 horas en una prueba de tolerancia oral a la glucosa
3. Pres = Presión arterial diastólica (mm Hg)
4. Piel = Espesor del pliegue cutáneo del tríceps (mm)
5. Insu = Insulina sérica de 2 horas (mu U / ml)
6. Masa = Índice de masa corporal (peso en kg / (altura en m) ^ 2)
7. Pedi = Función del pedigrí de la diabetes
8. Edad = Edad (años)

La etiqueta de clase representa si la persona no tiene diabetes (probado_negativo) o si la persona tiene diabetes (probado_positivo).

Los datos los tenemos disponibles en la carpeta `25_mastering_RF_XGBoost_practica` y, como en la práctica anterior, podemos utilizar la función `lecturaDatos` del fichero python `leer_datasets.py` para leerlos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from leer_datasets import lecturaDatos

# Definimos el nombre del dataset
dataset = 'Pima'

# Establecemos la ruta del train y el test
fileTr = 'pima-5-2tra.dat' 

fileTest='pima-5-2tst.dat'


# Leemos los datos con lecturaDatos 
datosTr, _ = lecturaDatos(fileTr)
datosTest, _ = lecturaDatos(fileTest)

# Creamos los DataFrames de train y test
df_train_original = pd.DataFrame(data = datosTr.data, columns = datosTr.feature_names)
df_train_original['Target'] = datosTr.target

df_test = pd.DataFrame(data = datosTest.data, columns = datosTest.feature_names)
df_test['Target'] = datosTest.target

print('- Dataset {}:'.format(dataset))
print('\t Número de atributos: ', len(datosTr.feature_names))
print('\t Número de elementos de cada clase en train: {}, {}'.format(np.bincount(datosTr.target)[0], np.bincount(datosTr.target)[1]))
print('\t Número de elementos de cada clase en test: {}, {}'.format(np.bincount(datosTest.target)[0], np.bincount(datosTest.target)[1]))

Matplotlib is building the font cache; this may take a moment.


- Dataset Pima:
	 Número de atributos:  8
	 Número de elementos de cada clase en train: 400, 214
	 Número de elementos de cada clase en test: 100, 54


El primer paso para empezar con los datos es realizar un análisis descriptivo de los mismos (EDA). 

En concreto, nos fijamos en que, si usamos la función `describe()` de Pandas, el valor mínimo de ciertas variables es 0. Pero en algunos casos, por su significado, ese valor no es posible. Por ejemplo, una persona no puede tener 0 de presión sanguínea.

Las variables con presencia de 0s erróneos son: ['Pres', 'Insu', 'Plas', 'Skin', 'Mass'].

Vamos a realizar una imputación de datos en las variables que tienen un valor 0 cuando no debería ser posible. 

Para ello, con el conjunto de entrenamiento, calculamos la media de los valores no nulos de esas variables e imputamos dichos valores en los ejemplos que tengan un 0. 
En cuanto al conjunto de test, realizamos la misma imputación pero utilizando los valores medios que hemos obtenido en el conjunto de train. 

In [2]:

# Importamos las librerías básicas a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import metrics, model_selection


In [3]:
#realizamos analisis EDA
df_test.describe(percentiles=None, include=None, exclude=None)

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Target
count,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000
mean,3.876623,121.597403,69.227273,18.064935,67.006494,31.301948,0.482883,33.363636,0.350649
std,3.740483,28.787597,20.551712,16.040665,88.001894,8.499569,0.353371,11.639362,0.478730
min,0.000000,65.000000,0.000000,0.000000,0.000000,0.000000,0.088000,21.000000,0.000000
25%,1.000000,102.000000,64.000000,0.000000,0.000000,26.775000,0.227750,23.250000,0.000000
50%,3.000000,117.000000,72.000000,19.500000,0.000000,31.800000,0.381500,30.000000,0.000000
75%,6.000000,140.500000,80.000000,31.000000,118.000000,36.300000,0.650500,40.000000,1.000000
max,17.000000,197.000000,110.000000,60.000000,330.000000,67.100000,2.288000,69.000000,1.000000


In [14]:
print(df_test.index)
print(df_test.columns)

RangeIndex(start=0, stop=154, step=1)
Index(['Preg', 'Plas', 'Pres', 'Skin', 'Insu', 'Mass', 'Pedi', 'Age',
       'Target'],
      dtype='object')


In [15]:
#Información de los 10 primeros registros
df_test.head(10)

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Target
0,4.0,146.0,78.0,0.0,0.0,38.5,0.520,67.0,1
1,15.0,136.0,70.0,32.0,110.0,37.1,0.153,43.0,1
2,10.0,101.0,86.0,37.0,0.0,45.6,1.136,38.0,1
3,1.0,168.0,88.0,29.0,0.0,35.0,0.905,52.0,1
4,5.0,96.0,74.0,18.0,67.0,33.6,0.997,43.0,0
5,2.0,127.0,58.0,24.0,275.0,27.7,1.600,25.0,0
6,10.0,162.0,84.0,0.0,0.0,27.7,0.182,54.0,0
7,13.0,76.0,60.0,0.0,0.0,32.8,0.180,41.0,0
8,5.0,126.0,78.0,27.0,22.0,29.6,0.439,40.0,0
9,4.0,76.0,62.0,0.0,0.0,34.0,0.391,25.0,0


In [16]:
#Información de los 10 últimos registros
df_test.tail(10)

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,Target
144,8.0,108.0,70.0,0.0,0.0,30.5,0.955,33.0,1
145,2.0,94.0,68.0,18.0,76.0,26.0,0.561,21.0,0
146,8.0,194.0,80.0,0.0,0.0,26.1,0.551,67.0,0
147,10.0,122.0,78.0,31.0,0.0,27.6,0.512,45.0,0
148,1.0,111.0,62.0,13.0,182.0,24.0,0.138,23.0,0
149,3.0,111.0,62.0,0.0,0.0,22.6,0.142,21.0,0
150,1.0,103.0,80.0,11.0,82.0,19.4,0.491,22.0,0
151,2.0,141.0,58.0,34.0,128.0,25.4,0.699,24.0,0
152,1.0,109.0,60.0,8.0,182.0,25.4,0.947,21.0,0
153,1.0,181.0,64.0,30.0,180.0,34.1,0.328,38.0,1


In [12]:
#Calculamos la media de las variables

#VAlor medio de Preg
print(df_test.Preg.mean())

#Valor medio de pres

print(df_test.Pres.mean())
       
#Valor medio de Skin
       
print (df_test.Skin.mean())     

#Valor medio de Insu
       
print (df_test.Insu.mean())  

#Valor medio de MAss
       
print (df_test.Mass.mean())     

#Valor medio de Target
       
print (df_test.Target.mean())  




3.8766233766233764
69.22727272727273
18.064935064935064
67.00649350649351
31.301948051948052
0.35064935064935066


In [22]:
df_test.index

RangeIndex(start=0, stop=154, step=1)

In [24]:
#Número de registros igual a cero
print(df_test.groupby('0').Preg.count())


KeyError: '0'

In [ ]:
#Misma información pero mostrado más bonito
pd.DataFrame(t.groupby('Embarked').PassengerId.count())

In [ ]:
#Función para cambiar '0' por la media
def cambiar(x):
    if x=='0':
        return '0'
    elif x=='female':
        return 'F'
    else:
        return x


## 25.2. Mastering Random Forest  <a class="anchor" id="2"></a>

Para trabajar con el ensemble Random Forest, vamos a utilizar la clases [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) de `sklearn.ensemble`. 

Para los propósitos de esta práctica, vamos a separar unos datos del conjunto de train con la intención de usarlos como conjunto de validación. Usa la función `train_test_split` para hacer esta separación. Usa 0.2 de `test_size` y 42 de `random_state`. 

En esta práctica, vamos a ir modificando el valor de 7 de los citados hiper-parámetros de uno en uno y dibujaremos curvas de rendimiento del modelo para entender cómo afecta cada uno de ellos. Los 7 que vamos a tener en cuenta son: 
1. max_depth
2. min_samples_split
3. max_leaf_nodes
4. min_samples_leaf
5. n_estimators
6. max_samples
7. max_features

### 1. max_depth
Número entero para la profundidad de los árboles. Su valor por defecto es None, que indica que los nodos se expanden hasta que todas las hojas son puras o hasta que todas contienen menos de `min_samples_split` ejemplos.

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 2 al 50 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica. 

Se observa que, conforme se aumenta la profundidad, los resultados de train mejoran pero los de test, después de mejorar al principio, empeoran rápidamente. Este hecho se debe al sobre-aprendizaje.

### 2. min_samples_split
Se trata del número mínimo de ejemplos que tiene que haber para poder dividir el nodo. Este parámetro admite un número entero, indicando el número exacto de ejemplos, o un float, indicando el porcentaje del número de ejemplos totales con el que queremos trabajar. Por defecto, toma el valor 2.

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 2 al 200 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica. 

Al aumentar el valor del hiper-parámetro `min_samples_split`, podemos ver claramente que, para valores bajos, existe una diferencia considerable en el rendimiento en train y validación. Pero a medida que aumenta el valor del parámetro, la diferencia entre ambas disminuye.

Cuando el valor del parámetro aumenta demasiado, ambos rendimientos decrecen. Esto se debe al hecho de que el requisito mínimo para dividir un nodo es tan alto que no se observan divisiones significativas. Como resultado, tenemos un problema de bias alto.

### 3. max_leaf_nodes
El número, entero, de nodos hoja que se permiten como máximo. Por defecto toma el valor None, indicando que no hay restricción.

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 2 al 200 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica. 

Podemos ver que cuando el valor del parámetro es pequeño, el árbol está desajustado y, a medida que aumenta el valor del parámetro, aumenta el rendimiento del árbol tanto en train como en validación. 

### 4. min_samples_leaf
El número mínimo de ejemplos que permitimos que haya en un nodo hoja. Admite números enteros o floats. 

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 1 al 100 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica.

Podemos ver que el modelo Random Forest está sobreajustado cuando el valor del parámetro es muy bajo, en valores intermedios se corrige y para valores altos empiezan problemas de bias alto. 

### 5. n_estimators
Hasta ahora hemos visto hiper-parámetros propios de los estimadores base. Pasamos a ver los del ensemble. Este en concreto es el número de estimadores base del ensemble. Por defecto, 100. 

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 1 al 200 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica.

El rendimiento crece al principio y pasa a mantenerse estable. Por lo tanto, podemos deducir que para este problema no es indispensable utilizar una gran cantidad de estimadores base. 

### 6. max_samples
Es el número que indica el número de ejemplos que tendrá cada re-muestreo. Este parámetro admite un número entero, indicando el número exacto de ejemplos, o un float, indicando el porcentaje del número de ejemplos totales con el que queremos trabajar. Por defecto, 1.0: utilizamos tantos ejemplos en el re-muestreo como tenemos en el conjunto original.

Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar 50 valores reales entre 0.01 y 0.99 (np.linspace) y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica.

El rendimiento crece rápido al principio y luego el error de validación se mantiene estable y el modelo se sobre-ajusta. La conclusión que sacamos es que no es necesario usar tantos ejemplos y se puede hacer un remuestreo de un número menor que el total de los datos para entrenar cada árbol. 

### 7. max_features
 El número de atributos que vamos a considerar en cada nodo. Admite los valores 'auto', 'sqrt', 'log2', un número entero, un float o None.
* 'auto': Es el valor por defecto e indica que el número de atributos en cada nodo sea la raíz cuadrada del número de atributos totales.
* 'sqrt': Como 'auto', indica que el número de atributos en cada nodo sea la raíz cuadrada del número de atributos totales.
* 'log2': Indica que el número de atributos en cada nodo sea el logaritmo en base 2 del número de atributos totales.
* Un número entero: Indica el número exacto de atributos. 
* Un float: Indica el porcentaje del número total de atributos que queremos utilizar en cada nodo. 
* None: Indica que el número de atributos sea igual que el número total de atributos.
    
Dibuja una gráfica evaluando el modelo en función del valor de este parámetro. En el eje X deben estar los valores del 1 al 8 y en el Y el F1-score. Se deben pintar la curva que describe el F1-score de train y de validación en la misma gráfica.    

En este caso también el rendimiento de validación es mayor cuando no se utilizan todos los atributos. Esto indica que no es necesario utilizar todos, pudiendo llegar a sobre-ajustar el modelo.

### Grid-search sobre los parámetros estudiados

Hemos visto cuáles podrían ser los valores óptimos de cada parámetro por separado, pero los hiper-parámetros no son independientes entre sí. El valor de cada uno de ellos influye en el resto. Realizaremos un grid-search para ver si con los valores obtenidos podemos mejorar el rendimiento del clasificador. 

Aunque no lo hayamos estudiado de forma específica, se puede introducir en el grid-search el parámetro `criterion`, que indica el tipo de árbol de decisión que va a usar el ensemble como clasificador base. 

Realiza un grid-search sobre los parámetros estudiados para ver cuál es la mejor combinación de parámetros para este modelo en estos datos. Utiliza el conjunto de train original para hacer validación cruzada. Al final, reporta el rendimiento en el conjunto de test. 

Intenta obtener el mejor resultado posible!

**Nota:** Si hacemos varias pruebas es conveniente fijar el random_state a un número para poder reproducir resultados.

## 25.3. Mastering XGBoost  <a class="anchor" id="3"></a>

XGBoost viene de "Extreme Gradient Boosting". Se trata de una librería optimizada y distribuida para aplicar algoritmos de Gradient Boosting ([aquí](https://xgboost.readthedocs.io/en/latest/index.html) su documentación). Además de los detalles de implementación que hacen que ahorremos recursos y ganemos en rapidez, una de las ventajas de xgboost es que da la opción de aplicar varias técnicas de regularización que nos pueden ser de utilidad.

Para utilizar esta implementación con Python, deberemos instalar el paquete `xgboost`. 

**Nota:** Para esta práctica debemos actualizar la versión del paquete `xgboost` que tenemos en el entorno.

Vamos a enumerar los parámetros de XGBoost y a optimizarlos con el mismo dataset que hemos usado para Random Forest. 

En este [link](https://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters) se puede encontrar una guía completa de los parámetros de este modelo, donde aparecen, además, sus valores por defecto. Muchos de ellos coinciden con los parámetros que vimos para Gradient Boosting. 

En general, el paquete xgboost es diferente al scikit-learn (por ejemplo los nombres de los parámetros varían). Sin embargo, existe un *wrapper* llamado XGBClassifier que imita las clases de scikit-learn, por lo que no se nos hará difícil usarlo. 

### Ajuste de parámetros

Los algoritmos como XGBoost tienen una gran cantidad de hiper-parámetros y optimizarlos no es una tarea fácil. No siempre se tiene tiempo ilimitado para realizar un grid-search de todos los parámetros simultáneamente, especialmente cuando trabajamos con datasets grandes, por lo tanto vamos a seguir otra metodología para ajustar los hiper-parámetros. 

Los parámetros que vamos a tener en cuenta para realizar el ajuste son:
* learning_rate 
* max_depth
* min_child_weight
* gamma
* subsample
* colsample_bytree
* reg_lambda
* reg_alpha

Para ajustarlos, seguiremos este esquema: 

1. Inicializamos el modelo y lo utilizamos como baseline. 
2. Comenzaremos ajustando la estructura de los árboles, con max_depth, min_child_weight.  
3. Una vez fijada la estructura, continuamos con los parámetros de sampleo de cada uno de los árboles (subsample y colsample_bytree).
4. Finalizamos con los parámetros que previenen el sobreaprendizaje (learning rate, gamma y los parámetros de regularización reg_lambda y reg_alpha).

El valor de los parámetros que vayamos obteniendo lo guardaremos en un diccionario `best_params` cuyas claves serán los nombres de los parámetros. 

Actualizamos el conjunto de datos e inicializamos el diccionario.

#### Ajuste del número de estimadores

Al ser XGBoost un modelo aditivo, donde cada estimador aporta mejoras al anterior, para optimizar el número de estimadores que utilizamos vamos a utilizar una técnica denominada **early stopping**, que consiste en entrenar el modelo cada vez con un estimador más e ir comprobando el error de validación. Cuando haya habido cierto número (establecido de antemano) de iteraciones en el que el error de validación no ha decrecido, se detiene el entrenamiento. 

Esto nos va a servir para establecer el número óptimo de árboles para la configuración de hiper-parámetros que consideremos. 

La técnica early stopping se puede utilizar directamente con la clase `XGBClassifier`, especificando en el constructor el número de iteraciones en las que el error de validación no debe decrecer para parar mediante el parámetro `early_stopping_rounds`. Como conjuntos de train y validación utilizaremos la separación de train y val que hemos realizado para ajustar el Random Forest.

Cada vez que entrenemos el modelo, ya sea directamente o mediante GridSearchCV, deberemos especificar cuál es el conjunto de validación que empleamos para hacer early stopping.

#### 1. Inicializar el modelo

Comenzamos el modelo con los siguientes parámetros: 

xgb = XGBClassifier(early_stopping_rounds=10, use_label_encoder=False, eval_metric='logloss')

Entrenamos el modelo, especificando en el parámetro `eval_set` el conjunto de validación (`eval_set = [(Xval, yval)]`) y observamos el rendimiento en test. 

#### 2. Ajustar los parámetros propios de los árboles

Esta tarea también la vamos a realizar en varios pasos. Empezamos por estos dos parámetros:

* `max_depth`: La máxima profundidad de los árboles. Mismo parámetro que en Gradient Boosting. A medida que crece, se incrementa la varianza. Valores típicos: 3-10. 

* `min_child_weight`: La suma mínima de los pesos de los ejemplos necesaria en una hoja. Si no se proporcionan pesos de los ejemplos, se considera que todos tienen el mismo. 

Realizamos un grid-search con los valores entre 3 y 10 para el primer parámetro y [8,9,10] para el segundo. Como scoring utilizamos el área bajo la curva ROC ('roc_auc'). Guardamos el valor que obtengamos en el diccionario `best_params` e introducimos el valor también en el modelo xgboost. 

#### 3. Ajustar los parámetros de sampleo

Los parámetros en los que nos fijamos ahora son: 
* `subsample`: Float que especifica la fracción de ejemplos con los que se aprenden los estimadores base. Valores típicos: 0.5-1.
* `colsample_bytree`: Float que especifica la fracción de atributos con los que se va a entrenar cada uno de los árboles. Similar al parámetro `max_features` de Gradient Boosting. También existe `colsample_bylevel`.

Los valores que vamos a probar para ambos parámetros son desde 0.1 a 1.0 (de 0.1 en 0.1).

#### 3. Ajustar los parámetros que previenen el sobreaprendizaje

Como primer paso, vamos a ajustar el factor de aprendizaje. Utilizamos los valores [0.01, 0.05, 0.1, 0.3, 1, 10]. 

Ahora vamos a fijarnos en el parámetro `gamma`:
* `gamma`: Umbral mínimo de la reducción del error al hacer la división de un nodo para poder llevar a cabo esa división. Por defecto, toma el valor 0. 

Realizamos un grid-search con 0 valores cogidos de forma uniforme entre 0.1 y 0.5.

Vamos a ajustar los parámetros propios de XGBoost que tienen que ver con la regularización:

* `reg_lambda`: Término de regularización L2. 

* `reg_alpha`: Término de regularización L1.

Vamos a probar los valores [0.01, 0.5, 1, 100] para `reg_lambda` y [1, 3, 5, 100] para `reg_alpha`.

¿Cuánto ha mejorado el accuracy desde el primero que hemos propuesto hasta este último?

Realiza las pruebas que consideres oportunas para mejorar la clasificación.

## 25.4. Pruebas libres adicionales <a class="anchor" id="4"></a>

Una vez estemos satisfechos con los resultados obtenidos en este dataset, es hora de escoger otro e intentar aplicar todo lo aprendido para conseguir un buen modelo. Una propuesta interesante es aplicar las técnicas de oversampling y undersampling estudiadas de forma conjunta a un ensemble para hacer predicciones en conjuntos de datos muy desbalanceados. En el siguiente [link](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/overview) hay una competición de Kaggle con un problema de clasificación binaria con esas características. 

